In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
import random
from pathlib import Path
import math
import time
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    accuracy_score,
    precision_recall_fscore_support
)
import pandas as pd

# Configurations
DATA_ROOT = "/kaggle/input/plantvillage"
IMG_SIZE = 256
PATCH_SIZE = 16
NUM_CLASSES = 38
BATCH_SIZE = 64
EPOCHS = 25
LEARNING_RATE = 0.0003
WARMUP_EPOCHS = 5
WEIGHT_DECAY = 0.05
MIN_LR = 1e-6
EARLY_STOP_PATIENCE = 7
MIN_EPOCHS = 12
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device: {DEVICE}")
print(f"Configuration:")
print(f"  Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS} (max, with early stopping)")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Early Stop Patience: {EARLY_STOP_PATIENCE}")

# Dataset
class PlantVillageDataset(Dataset):
    def __init__(self, data_root, transform=None, img_size=IMG_SIZE):
        self.data_root = Path(data_root)
        self.transform = transform
        self.img_size = img_size
        
        possible_paths = [
            self.data_root / "PlantVillage",
            self.data_root / "plantvillage",
            self.data_root / "color",
            self.data_root / "PlantVillageDataset"
        ]
        
        actual_path = None
        for path in possible_paths:
            if path.exists():
                actual_path = path
                break
        
        if actual_path is None:
            for item in self.data_root.iterdir():
                if item.is_dir() and "train" in str(item).lower():
                    actual_path = item
                    break
        
        if actual_path is None:
            raise FileNotFoundError(f"Could not find PlantVillage data in {data_root}")
        
        train_path = actual_path / "train"
        if train_path.exists():
            actual_path = train_path
        
        self.base_path = actual_path
        self.image_paths = []
        self.labels = []
        self.class_names = []
        self.class_to_idx = {}
        
        class_dirs = sorted([d for d in self.base_path.iterdir() if d.is_dir()])
        
        if not class_dirs:
            for item in self.base_path.iterdir():
                if item.is_dir():
                    class_dirs = sorted([d for d in item.iterdir() if d.is_dir()])
                    if class_dirs:
                        self.base_path = item
                        break
        
        for idx, class_dir in enumerate(class_dirs):
            class_name = class_dir.name
            self.class_to_idx[class_name] = idx
            self.class_names.append(class_name)
            
            image_files = []
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
                image_files.extend(list(class_dir.glob(ext)))
            
            for img_path in image_files:
                self.image_paths.append(img_path)
                self.labels.append(idx)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
        else:
            img = T.Compose([
                T.Resize((self.img_size, self.img_size)),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])(img)
        
        return img, label


# Data augmentation
train_transform = T.Compose([
    T.Resize((IMG_SIZE + 32, IMG_SIZE + 32)),
    T.RandomCrop(IMG_SIZE),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomVerticalFlip(p=0.15),
    T.RandomRotation(15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    T.RandomErasing(p=0.2, scale=(0.02, 0.1))
])

val_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset loading

dataset = PlantVillageDataset(DATA_ROOT, transform=train_transform)
print(f"Total images: {len(dataset):,}")
print(f"Number of classes: {len(dataset.class_names)}")

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

val_dataset.dataset.transform = val_transform
test_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          num_workers=4, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=4, pin_memory=True, persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=4, pin_memory=True, persistent_workers=True)

print(f"Train: {len(train_dataset):,} | Val: {len(val_dataset):,} | Test: {len(test_dataset):,}")


# MODEL ARCHITECTURE
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=256, patch_size=16, in_channels=3, embed_dim=512):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches + 1, embed_dim))
    
    def forward(self, x):
        B = x.shape[0]
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)
        
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim=512, num_heads=8, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5
        
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.attn_drop = nn.Dropout(dropout)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.proj_drop = nn.Dropout(dropout)
    
    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, dropout=0.1):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim=512, num_heads=8, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadAttention(embed_dim, num_heads, dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        
        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = MLP(embed_dim, mlp_hidden_dim, dropout=dropout)
    
    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class ImprovedVisionTransformer(nn.Module):
    def __init__(self, img_size=256, patch_size=16, in_channels=3, num_classes=38,
                 embed_dim=512, depth=8, num_heads=8, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        self.pos_drop = nn.Dropout(dropout)
        
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(depth)
        ])
        
        self.norm = nn.LayerNorm(embed_dim)
        
        self.head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim // 2, num_classes)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.trunc_normal_(self.patch_embed.cls_token, std=0.02)
        nn.init.trunc_normal_(self.patch_embed.pos_embed, std=0.02)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.LayerNorm):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
    
    def forward(self, x):
        x = self.patch_embed(x)
        x = self.pos_drop(x)
        
        for block in self.blocks:
            x = block(x)
        
        x = self.norm(x)
        x = x[:, 0]
        x = self.head(x)
        return x


# Model initialization
model = ImprovedVisionTransformer(
    img_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    num_classes=NUM_CLASSES,
    embed_dim=512,
    depth=8,
    num_heads=8,
    mlp_ratio=4.0,
    dropout=0.15
).to(DEVICE)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")


# Training setup
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

def get_lr_scheduler(optimizer, warmup_epochs, total_epochs, min_lr=1e-6):
    def lr_lambda(epoch):
        if epoch < warmup_epochs:
            return (epoch + 1) / warmup_epochs
        else:
            progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
            cosine_decay = 0.5 * (1 + math.cos(math.pi * progress))
            return max(cosine_decay, min_lr / LEARNING_RATE)
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

scheduler = get_lr_scheduler(optimizer, WARMUP_EPOCHS, EPOCHS, MIN_LR)


# Training loop
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

train_losses, train_accs = [], []
val_losses, val_accs = [], []
best_val_acc = 0.0
best_epoch = 0
patience_counter = 0

training_start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    
    # Training
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1:02d}/{EPOCHS}", leave=False)
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        if torch.isnan(loss):
            print(f"\nWarning: NaN loss at epoch {epoch+1}, batch {batch_idx}")
            continue
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
        optimizer.step()
        
        epoch_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if batch_idx % 20 == 0:
            pbar.set_postfix({
                'Loss': f"{epoch_loss/(batch_idx+1):.4f}",
                'Acc': f"{100.*correct/total:.2f}%"
            })
    
    train_loss = epoch_loss / len(train_loader)
    train_acc = 100. * correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    val_loss = val_loss / len(val_loader)
    val_acc = 100. * correct / total
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    epoch_time = time.time() - epoch_start_time
    
    print(f"Epoch {epoch+1:02d}/{EPOCHS}: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}% | "
          f"LR: {current_lr:.6f} | Time: {epoch_time/60:.1f}m")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
        patience_counter = 0
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'train_acc': train_acc,
            'class_names': dataset.class_names,
            'train_losses': train_losses,
            'train_accs': train_accs,
            'val_losses': val_losses,
            'val_accs': val_accs
        }, "/kaggle/working/best_model.pth")
        
        print(f"  ✓ New best! Val Acc: {val_acc:.2f}%")
    else:
        patience_counter += 1
        print(f"  No improvement ({patience_counter}/{EARLY_STOP_PATIENCE})")
    
    # Early stopping
    if patience_counter >= EARLY_STOP_PATIENCE and epoch >= MIN_EPOCHS:
        print(f"\n⏹ Early stopping at epoch {epoch+1}")
        print(f"  Best val acc: {best_val_acc:.2f}% at epoch {best_epoch}")
        break

total_training_time = time.time() - training_start_time

print(f"Training completed in {total_training_time/60:.1f} minutes")
print(f"Best validation accuracy: {best_val_acc:.2f}% at epoch {best_epoch}")



# EVALUATION - COLLECT PREDICTIONS

checkpoint = torch.load("/kaggle/working/best_model.pth")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

class_names = dataset.class_names

def get_predictions(model, data_loader, device):
    """Get all predictions and true labels"""
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())
    
    return np.array(all_labels), np.array(all_preds), np.array(all_probs)

print("\nCollecting predictions...")
test_labels, test_preds, test_probs = get_predictions(model, test_loader, DEVICE)
val_labels, val_preds, val_probs = get_predictions(model, val_loader, DEVICE)


# CALCULATE METRICS
print("\nCalculating metrics...")

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average='weighted', zero_division=0
    )
    precision_per_class, recall_per_class, f1_per_class, support_per_class = \
        precision_recall_fscore_support(y_true, y_pred, average=None, zero_division=0)
    
    return {
        'accuracy': accuracy,
        'precision': precision * 100,
        'recall': recall * 100,
        'f1': f1 * 100,
        'precision_per_class': precision_per_class,
        'recall_per_class': recall_per_class,
        'f1_per_class': f1_per_class,
        'support_per_class': support_per_class
    }

test_metrics = calculate_metrics(test_labels, test_preds)
val_metrics = calculate_metrics(val_labels, val_preds)

print("\n📊 TEST SET METRICS:")
print(f"  Accuracy:  {test_metrics['accuracy']:.2f}%")
print(f"  Precision: {test_metrics['precision']:.2f}%")
print(f"  Recall:    {test_metrics['recall']:.2f}%")
print(f"  F1-Score:  {test_metrics['f1']:.2f}%")

print("\n📊 VALIDATION SET METRICS:")
print(f"  Accuracy:  {val_metrics['accuracy']:.2f}%")
print(f"  Precision: {val_metrics['precision']:.2f}%")
print(f"  Recall:    {val_metrics['recall']:.2f}%")
print(f"  F1-Score:  {val_metrics['f1']:.2f}%")


# CONFUSION MATRIX
print("\nGenerating confusion matrices...")

def plot_confusion_matrix(y_true, y_pred, class_names, title, save_path):
    cm = confusion_matrix(y_true, y_pred)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    plt.figure(figsize=(20, 18))
    sns.heatmap(cm_percent, annot=True, fmt='.1f', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Percentage (%)'}, linewidths=0.5)
    
    plt.title(title, fontsize=16, fontweight='bold', pad=20)
    plt.ylabel('True Label', fontsize=14)
    plt.xlabel('Predicted Label', fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return cm

test_cm = plot_confusion_matrix(test_labels, test_preds, class_names,
    "Test Set Confusion Matrix", "/kaggle/working/confusion_matrix_test.png")

val_cm = plot_confusion_matrix(val_labels, val_preds, class_names,
    "Validation Set Confusion Matrix", "/kaggle/working/confusion_matrix_val.png")

print("  ✓ Confusion matrices saved")


# Training curves
print("\nGenerating training curves...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss
axes[0, 0].plot(train_losses, label='Training Loss', linewidth=2.5, color='#e74c3c', alpha=0.8)
axes[0, 0].plot(val_losses, label='Validation Loss', linewidth=2.5, color='#3498db', alpha=0.8)
axes[0, 0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Training & Validation Loss', fontsize=14, fontweight='bold', pad=15)
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(train_accs, label='Training Accuracy', linewidth=2.5, color='#e74c3c', alpha=0.8)
axes[0, 1].plot(val_accs, label='Validation Accuracy', linewidth=2.5, color='#3498db', alpha=0.8)
axes[0, 1].axhline(y=75, color='green', linestyle='--', linewidth=2, alpha=0.6, label='75% Target')
axes[0, 1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Training & Validation Accuracy', fontsize=14, fontweight='bold', pad=15)
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Accuracy progression
epochs = range(1, len(train_accs) + 1)
axes[1, 0].scatter(epochs, train_accs, label='Training', alpha=0.6, s=80, color='#e74c3c')
axes[1, 0].scatter(epochs, val_accs, label='Validation', alpha=0.6, s=80, color='#3498db')
axes[1, 0].plot(epochs, train_accs, alpha=0.3, color='#e74c3c')
axes[1, 0].plot(epochs, val_accs, alpha=0.3, color='#3498db')
axes[1, 0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Accuracy Progression', fontsize=14, fontweight='bold', pad=15)
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Overfitting gap
gap = np.array(train_accs) - np.array(val_accs)
axes[1, 1].plot(epochs, gap, linewidth=2.5, color='#9b59b6', alpha=0.8)
axes[1, 1].axhline(y=0, color='green', linestyle='--', linewidth=2, alpha=0.6, label='No Gap')
axes[1, 1].axhline(y=10, color='orange', linestyle='--', linewidth=2, alpha=0.6, label='10% Gap')
axes[1, 1].fill_between(epochs, 0, gap, alpha=0.3, color='#9b59b6')
axes[1, 1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Accuracy Gap (%)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Overfitting Analysis', fontsize=14, fontweight='bold', pad=15)
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/training_curves.png', dpi=300, bbox_inches='tight')
plt.close()

print("  ✓ Training curves saved")


# PER-CLASS PERFORMANCE
print("\nGenerating per-class performance...")

df = pd.DataFrame({
    'Class': class_names,
    'Precision': test_metrics['precision_per_class'] * 100,
    'Recall': test_metrics['recall_per_class'] * 100,
    'F1-Score': test_metrics['f1_per_class'] * 100,
    'Support': test_metrics['support_per_class']
})
df = df.sort_values('F1-Score', ascending=True)

fig, axes = plt.subplots(1, 2, figsize=(20, 12))

y_pos = np.arange(len(df))

axes[0].barh(y_pos - 0.2, df['Precision'], 0.2, label='Precision', alpha=0.8, color='#3498db')
axes[0].barh(y_pos, df['Recall'], 0.2, label='Recall', alpha=0.8, color='#e74c3c')
axes[0].barh(y_pos + 0.2, df['F1-Score'], 0.2, label='F1-Score', alpha=0.8, color='#2ecc71')
axes[0].set_yticks(y_pos)
axes[0].set_yticklabels([name[:30] for name in df['Class']], fontsize=8)
axes[0].set_xlabel('Score (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Per-Class Performance', fontsize=14, fontweight='bold', pad=15)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3, axis='x')
axes[0].set_xlim(0, 100)

axes[1].barh(y_pos, df['Support'], alpha=0.7, color='#95a5a6')
axes[1].set_yticks(y_pos)
axes[1].set_yticklabels([name[:30] for name in df['Class']], fontsize=8)
axes[1].set_xlabel('Number of Samples', fontsize=12, fontweight='bold')
axes[1].set_title('Sample Distribution', fontsize=14, fontweight='bold', pad=15)
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('/kaggle/working/per_class_metrics.png', dpi=300, bbox_inches='tight')
plt.close()

print("  ✓ Per-class metrics saved")


# SAMPLE PREDICTIONS
print("\nGenerating sample predictions...")

sample_indices = random.sample(range(len(test_dataset)), 12)

fig, axes = plt.subplots(3, 4, figsize=(20, 15))
axes = axes.flatten()

for i, idx in enumerate(sample_indices):
    image, true_label = test_dataset[idx]
    
    img_display = image.clone().permute(1, 2, 0).numpy()
    img_display = img_display * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img_display = np.clip(img_display, 0, 1)
    
    with torch.no_grad():
        image_tensor = image.unsqueeze(0).to(DEVICE)
        outputs = model(image_tensor)
        probs = torch.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probs, 1)
    
    true_class = class_names[true_label]
    pred_class = class_names[predicted.item()]
    conf = confidence.item()
    
    axes[i].imshow(img_display)
    
    if true_class == pred_class:
        color = 'green'
        marker = '✓'
        border_color = '#2ecc71'
    else:
        color = 'red'
        marker = '✗'
        border_color = '#e74c3c'
    
    true_display = true_class[:25] + '...' if len(true_class) > 25 else true_class
    pred_display = pred_class[:25] + '...' if len(pred_class) > 25 else pred_class
    
    title = f"{marker}\nTrue: {true_display}\nPred: {pred_display}\nConf: {conf:.2%}"
    axes[i].set_title(title, color=color, fontsize=10, fontweight='bold', pad=10)
    axes[i].axis('off')
    
    for spine in axes[i].spines.values():
        spine.set_edgecolor(border_color)
        spine.set_linewidth(4)
        spine.set_visible(True)

plt.suptitle(f'Sample Predictions (Test Acc: {test_metrics["accuracy"]:.2f}%)', 
             fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('/kaggle/working/sample_predictions.png', dpi=300, bbox_inches='tight')
plt.close()

print("  ✓ Sample predictions saved")


# CLASSIFICATION REPORT
print("\nGenerating classification report...")

report = classification_report(test_labels, test_preds, target_names=class_names, 
                               digits=3, zero_division=0)
with open('/kaggle/working/classification_report.txt', 'w') as f:
    f.write(report)

report_dict = classification_report(test_labels, test_preds, target_names=class_names, 
                                   output_dict=True, zero_division=0)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv('/kaggle/working/classification_report.csv')

print("  ✓ Classification report saved")

# ============================================================
# METRICS SUMMARY
# ============================================================
summary_data = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Test Set': [
        f"{test_metrics['accuracy']:.2f}%",
        f"{test_metrics['precision']:.2f}%",
        f"{test_metrics['recall']:.2f}%",
        f"{test_metrics['f1']:.2f}%"
    ],
    'Validation Set': [
        f"{val_metrics['accuracy']:.2f}%",
        f"{val_metrics['precision']:.2f}%",
        f"{val_metrics['recall']:.2f}%",
        f"{val_metrics['f1']:.2f}%"
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('/kaggle/working/metrics_summary.csv', index=False)


# FINAL SUMMARY
print("EVALUATION COMPLETE!")

print("\n📁 SAVED FILES:")
print("  1. best_model.pth - Best model checkpoint")
print("  2. confusion_matrix_test.png")
print("  3. confusion_matrix_val.png")
print("  4. training_curves.png")
print("  5. per_class_metrics.png")
print("  6. sample_predictions.png")
print("  7. classification_report.txt")
print("  8. classification_report.csv")
print("  9. metrics_summary.csv")

print("\n✅ REQUIREMENT CHECK:")
if test_metrics['accuracy'] >= 75.0:
    print(f"  ✓ Test Accuracy: {test_metrics['accuracy']:.2f}% ≥ 75% - TARGET MET! 🎉")
else:
    print(f"  ✗ Test Accuracy: {test_metrics['accuracy']:.2f}% < 75%")
    print(f"  Gap: {75.0 - test_metrics['accuracy']:.2f}%")

print("\n📊 FINAL METRICS SUMMARY:")
print(f"  Training Time:     {total_training_time/60:.1f} minutes")
print(f"  Epochs Trained:    {len(train_accs)}")
print(f"  Best Epoch:        {best_epoch}")
print(f"  Best Val Acc:      {best_val_acc:.2f}%")
print(f"  Test Accuracy:     {test_metrics['accuracy']:.2f}%")
print(f"  Test Precision:    {test_metrics['precision']:.2f}%")
print(f"  Test Recall:       {test_metrics['recall']:.2f}%")
print(f"  Test F1-Score:     {test_metrics['f1']:.2f}%")

print("\n🏆 TOP 5 BEST PERFORMING CLASSES:")
top5 = df.tail(5)[['Class', 'F1-Score']].to_string(index=False, header=False)
print(top5)

print("\n⚠️ BOTTOM 5 WORST PERFORMING CLASSES:")
bottom5 = df.head(5)[['Class', 'F1-Score']].to_string(index=False, header=False)
print(bottom5)

print("ALL DONE!")

Device: cuda
Configuration:
  Image Size: 256x256
  Batch Size: 64
  Epochs: 25 (max, with early stopping)
  Learning Rate: 0.0003
  Early Stop Patience: 7

LOADING DATASET
Total images: 43,444
Number of classes: 38
Train: 34,755 | Val: 4,344 | Test: 4,345
Total parameters: 25,887,014

STARTING TRAINING


Epoch 01/25: Train Loss: 2.1209, Train Acc: 53.34% | Val Loss: 1.7698, Val Acc: 60.70% | LR: 0.000120 | Time: 4.9m
  ✓ New best! Val Acc: 60.70%


Epoch 02/25: Train Loss: 1.3605, Train Acc: 76.67% | Val Loss: 1.1911, Val Acc: 81.10% | LR: 0.000180 | Time: 4.8m
  ✓ New best! Val Acc: 81.10%


Epoch 03/25: Train Loss: 1.1638, Train Acc: 82.74% | Val Loss: 1.1392, Val Acc: 82.71% | LR: 0.000240 | Time: 4.8m
  ✓ New best! Val Acc: 82.71%


Epoch 04/25: Train Loss: 1.0911, Train Acc: 85.36% | Val Loss: 1.2997, Val Acc: 78.13% | LR: 0.000300 | Time: 4.8m
  No improvement (1/7)


...

Epoch 22/25: Train Loss: 0.6833, Train Acc: 99.86% | Val Loss: 0.7209, Val Acc: 98.32% | LR: 0.000016 | Time: 4.8m
  ✓ New best! Val Acc: 98.32%


Epoch 23/25: Train Loss: 0.6820, Train Acc: 99.89% | Val Loss: 0.7330, Val Acc: 97.79% | LR: 0.000007 | Time: 4.8m
  No improvement (1/7)


Epoch 24/25: Train Loss: 0.6806, Train Acc: 99.94% | Val Loss: 0.7335, Val Acc: 97.74% | LR: 0.000002 | Time: 4.8m
  No improvement (2/7)


Epoch 25/25: Train Loss: 0.6802, Train Acc: 99.95% | Val Loss: 0.7305, Val Acc: 97.91% | LR: 0.000001 | Time: 4.8m
  No improvement (3/7)
Training completed in 120.9 minutes
Best validation accuracy: 98.32% at epoch 22




Calculating metrics...

📊 TEST SET METRICS:
  Accuracy:  97.95%
  Precision: 97.98%
  Recall:    97.95%
  F1-Score:  97.95%

📊 VALIDATION SET METRICS:
  Accuracy:  98.32%
  Precision: 98.35%
  Recall:    98.32%
  F1-Score:  98.32%

Generating confusion matrices...
  ✓ Confusion matrices saved

Generating training curves...
  ✓ Training curves saved

Generating per-class performance...
  ✓ Per-class metrics saved

Generating sample predictions...
  ✓ Sample predictions saved

Generating classification report...
  ✓ Classification report saved
EVALUATION COMPLETE!

📁 SAVED FILES:
  1. best_model.pth - Best model checkpoint
  2. confusion_matrix_test.png
  3. confusion_matrix_val.png
  4. training_curves.png
  5. per_class_metrics.png
  6. sample_predictions.png
  7. classification_report.txt
  8. classification_report.csv
  9. metrics_summary.csv

✅ REQUIREMENT CHECK:
  ✓ Test Accuracy: 97.95% ≥ 75% - TARGET MET! 🎉

📊 FINAL METRICS SUMMARY:
  Training Time:     120.9 minutes
  Epochs Tr